In [ ]:
import os
from pathlib import Path

import syft_rds as sy

import syft_flwr

## Set up

In [ ]:
EXAMPLE = "quickstart-pytorch"
# EXAMPLE = "embedded-devices"

if EXAMPLE == "embedded-devices":
    SYFTBOX_DATASET_NAME = "fashionmnist"
elif EXAMPLE == "quickstart-pytorch":
    SYFTBOX_DATASET_NAME = "cifar10"

PROJECT_DIR = Path(os.getcwd()).parent.parent.absolute()
FLWR_PROJECT_PATH = PROJECT_DIR / "examples" / EXAMPLE
SIM_DIR = PROJECT_DIR / ".sim"
CONFIG_DIR = SIM_DIR / ".config"

DO1 = "yash@openmined.org"
DO2 = "rasswanth@openmined.org"
DS = "khoa@openmined.org"

DS_CONFIG = f"{CONFIG_DIR}/{DS}.json"

!just dump-sim-config {DS}

In [ ]:
# do_syftbox_client = Client.load(DS_CONFIG)
# event_server = create_app(client=do_syftbox_client)
# event_server.start()

In [ ]:
do_client_1 = sy.init_session(host=DO1, syftbox_client_config_path=DS_CONFIG)
assert do_client_1.host == DO1
assert do_client_1.email == DS
assert not do_client_1.is_admin

do_client_2 = sy.init_session(host=DO2, syftbox_client_config_path=DS_CONFIG)
assert do_client_2.host == DO2
assert do_client_2.email == DS
assert not do_client_2.is_admin

datasites = [do_client_1, do_client_2]

In [ ]:
dataset1 = do_client_1.dataset.get(name=SYFTBOX_DATASET_NAME)
dataset1.describe()

In [ ]:
dataset2 = do_client_2.dataset.get(name=SYFTBOX_DATASET_NAME)
dataset2.describe()

In [ ]:
try:
    syft_flwr.bootstrap(FLWR_PROJECT_PATH, aggregator=DS, datasites=[DO1, DO2])
    print("Project bootstrapped successfully")
except Exception as e:
    print(f"Boostrap failed with err: {e}")

In [ ]:
# Job submission
! rm -rf {FLWR_PROJECT_PATH / ".venv"} {FLWR_PROJECT_PATH / "pytorch" / "__pycache__"} {FLWR_PROJECT_PATH / "pytorch" / "weights"} 

for client in datasites:
    job = client.jobs.submit(
        name="Syft Flower Experiment",
        description="Syft Flower Federated Learning Experiment",
        user_code_path=FLWR_PROJECT_PATH,
        dataset_name=SYFTBOX_DATASET_NAME,
        # runtime="./dockerfile", # path to a local dockerfile
        tags=["federated", "learning", "syft", "flwr"],
        entrypoint="main.py",
    )
    print(job)

## Run the server side code (Self Submit - TODO)

In [ ]:
# ds_client = sy.init_session(host=DS, syftbox_client_config_path=DS_CONFIG)

# self_job = ds_client.jobs.submit(
#         name="Syft Flower Experiment",
#         description="Syft Flower Federated Learning Experiment",
#         user_code_path=FLWR_PROJECT_PATH,
#         dataset_name=SYFTBOX_DATASET_NAME,
#         # runtime="./dockerfile", # path to a local dockerfile
#         tags=["federated", "learning", "syft", "flwr"],
#         entrypoint="main.py"
# )
# self_job

In [ ]:
# ds_client.run_private(self_job)

## Run simulation using the `main.py` file directly

In [ ]:
os.environ["SYFTBOX_CLIENT_CONFIG_PATH"] = str(DS_CONFIG)
!uv run {str(FLWR_PROJECT_PATH / "main.py")} --active

In [ ]:
# syft_flwr.view_server_logs(name=FLWR_PROJECT_PATH.name)

## TODO - Run simulation (locally on DS's machine)


In [ ]:
# syft_flwr.run_simulation(
#     path=FLWR_PROJECT_PATH,
#     mock_paths=[
#         do_client.datasets.get(name=SYFTBOX_DATASET_NAME).mock_path
#         for do_client in datasites
#     ],
#     num_datasites=len(datasites),
# )